In [1]:
# import requests
from __future__ import print_function    # (at top of module)
import time
import sys
import os
import pandas as pd
import numpy as np
import pickle

PROJ_ROOT = os.path.join(os.getcwd(), os.pardir)

%load_ext watermark
%watermark -a "Thomas Turner" -d -t -v -p numpy,pandas -g

Thomas Turner 07/30/2016 18:08:20 

CPython 3.5.2
IPython 5.0.0

numpy 1.11.1
pandas 0.18.1
Git hash: de5f8a4a58c61e5bd695a9f7c5e38a32dd9497b2


In [2]:
# load environment variables from .env file using dotenv.
from dotenv import load_dotenv
dotenv_path = os.path.join(PROJ_ROOT, '.env')
load_dotenv(dotenv_path)

# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(PROJ_ROOT, 'src')
sys.path.append(src_dir)

# import my methods from the source code
%aimport data.spotipy_functions
from data.spotipy_functions import *

In [3]:
# run this to generate a fresh 100 track recomendations from a seed
trackset = get_new_recs_and_feats(['1ohelc8DR2NNpUGukeCnGY'],100)
trackset = band_BPMs(trackset,80,170)

# run this to SAVE trackset to file
# trackset.to_pickle(os.path.join(PROJ_ROOT,'data','interim','recommends_data_proto1.pkl'))

# run this to LOAD trackset from file
# trackset = pd.read_pickle(os.path.join(PROJ_ROOT,'data','interim','recommends_data_proto1.pkl'))


In [17]:
# trackset.set_index(['id'])  <--- maybe not a good idea.

In [4]:
gui_cols = ['artist_name','track_name','id']
trained_so_far = 5
trackset.head(trained_so_far)[gui_cols]

,artist_name,track_name,id
0,Egal 3,La Tara - Original mix,2wi8ZPChQn2uF1vXeSpZsb
1,UNER,Positive,4680yXt48wO0n3VZ8EQJT7
2,Paul Loraine,Players Paradise,2OjuS64CIMgAZmh93aAPTq
3,Alex Arnout,Hold on (original) - Original,3wmHxvIRLePmsSjQXcYdmI
4,Gorge,LA angels - Audiojack remix,6ukk3EODVnVwP4Vci8B2jl


In [5]:

# np.set_printoptions(precision=4)
# np.set_printoptions(suppress=True)

useful_features = ['acousticness','danceability','energy','speechiness','tempo','valence']
X = trackset.head(trained_so_far)[useful_features]
Y = [0,1,0,1,0]
w = np.array([1,1,1,1,1])

In [6]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y, sample_weight=w)

GaussianNB()

In [7]:
pd.options.display.float_format = '{:,.3f}'.format
predicts = pd.DataFrame(clf.predict_proba(trackset.tail(trackset.shape[0] - trained_so_far)[useful_features]))
predicts.columns = ['P_reject','P_accept']
predicts_labeled = trackset.tail(trackset.shape[0] - trained_so_far)[gui_cols].reset_index()
predicts_labeled['P_accept'] = predicts.P_accept
predicts_labeled.sort_values(by=['P_accept'], ascending=False)

,index,artist_name,track_name,id,P_accept
94,99,Martin Patino,Michal - Original Mix,7MoNTLNK1iMJSGX8wMEclf,1.000
40,45,UNER,Monday Afternoon,1p3itkEGikqa7aoqSzNiuK,1.000
80,85,Martin Patino,Foundation,3qs14YmEYwpU02hbTMyQm1,1.000
28,33,Olderic,Oasis - Original,4fG8vgdqKIxGsAOC37Pzpi,1.000
29,34,Paul Loraine,Making A Difference - Helmut Ebritsch Remix,0K7tkb419yKEYvpejGQwbK,1.000
32,37,Djebali,Reputation - Original Mix,4htidg6DUF02QLjk7RcxBb,1.000
72,77,WestBoy,S L I V A - Chaim Remix,7LUOTO5NAplsSaJno8xejY,1.000
38,43,Martin Patino,Hommage a la Sodomie,2GM1THrQFB6ILVgcX1vAbB,1.000
65,70,Gavin Herlihy,Krypton Factor - Delano's Deep Space Mix,2QMQz19Dfeaiysicf2C5KP,1.000
18,23,UNER,Surfering,7lYs5TJwfKSVHYMBZdW4K6,1.000


In [19]:
import seaborn as sb
#sb.pairplot(trackset[useful_features])

{'artist_name': {0: 'Tom Budden', 1: 'Gabriele Baldi'},
 'id': {0: '0yrgY6xIMIl6bGx7HijcPC', 1: '6IEdf9jzyS12qesXqg48Mt'},
 'track_name': {0: 'The Tree Dance', 1: 'Sixty'}}

In [41]:
get_tracks_details(['0yrgY6xIMIl6bGx7HijcPC','6IEdf9jzyS12qesXqg48Mt'])

{'artist_name': ['Tom Budden', 'Gabriele Baldi'],
 'id': ['0yrgY6xIMIl6bGx7HijcPC', '6IEdf9jzyS12qesXqg48Mt'],
 'track_name': ['The Tree Dance', 'Sixty']}

In [49]:
sp = spotipy.Spotify()
input_tracks = ['0yrgY6xIMIl6bGx7HijcPC','6IEdf9jzyS12qesXqg48Mt']
results = sp.tracks(input_tracks)
results_df = pd.DataFrame(columns=('id','track_name','artist_name'))
for i, track in enumerate(results['tracks']):
    results_df.loc[i] = [track['id'],track['name'],track['artists'][0]['name']]
results_df

,id,track_name,artist_name
0,0yrgY6xIMIl6bGx7HijcPC,The Tree Dance,Tom Budden
1,6IEdf9jzyS12qesXqg48Mt,Sixty,Gabriele Baldi


In [52]:
a = ['0yrgY6xIMIl6bGx7HijcPC','6IEdf9jzyS12qesXqg48Mt']
get_tracks_details(a)

,id,track_name,artist_name
0,0yrgY6xIMIl6bGx7HijcPC,The Tree Dance,Tom Budden
1,6IEdf9jzyS12qesXqg48Mt,Sixty,Gabriele Baldi


In [53]:
get_recommends_from_seed(a,5)

,id,track_name,artist_name
0,6LI4X0kGR6FmcEHla0eonf,Dus,Maximiljan
1,4TqNE57G4gZRtyNsALTGvg,Hey Senor! - Original Mix,Luca M
2,3iMtZTXlJq0VWJLjeKfXnC,Fruitcake - Logiztik Sounds & Lupo Remix,Anil Chawla
3,4q2Hl9niW88FBpPpL3wdp8,The Queen,Strict Border
4,4PcwjNeZAfpTlDBExqoC6r,Don't Say A Word - Roberto Palmero Remix,Roberto Palmero


In [57]:
trackset

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,speechiness,tempo,time_signature,track_href,type,uri,valence,track_name,artist_name,state
0,0.003,https://api.spotify.com/v1/audio-analysis/038B...,0.805,454915,0.487,038BZXHIetx7G12SVsLwvT,0.865,7,0.140,-10.023,...,0.062,120.004,4,https://api.spotify.com/v1/tracks/038BZXHIetx7...,audio_features,spotify:track:038BZXHIetx7G12SVsLwvT,0.633,In My Soul - Asadinho Remix,Alex Arnout,0.000
1,0.025,https://api.spotify.com/v1/audio-analysis/4qSY...,0.818,352821,0.368,4qSYYLwNU1aY45wotO0ylP,0.854,1,0.486,-12.547,...,0.050,117.004,4,https://api.spotify.com/v1/tracks/4qSYYLwNU1aY...,audio_features,spotify:track:4qSYYLwNU1aY45wotO0ylP,0.463,Vesper - NTFO Remix,Tom Budden,0.000
2,0.118,https://api.spotify.com/v1/audio-analysis/1p3i...,0.804,469015,0.482,1p3itkEGikqa7aoqSzNiuK,0.893,10,0.114,-15.274,...,0.051,124.041,4,https://api.spotify.com/v1/tracks/1p3itkEGikqa...,audio_features,spotify:track:1p3itkEGikqa7aoqSzNiuK,0.609,Monday Afternoon,UNER,0.000
3,0.022,https://api.spotify.com/v1/audio-analysis/6JHQ...,0.806,417375,0.956,6JHQ5BmfyfYChzM9tr8F8V,0.863,5,0.104,-6.376,...,0.070,124.000,4,https://api.spotify.com/v1/tracks/6JHQ5BmfyfYC...,audio_features,spotify:track:6JHQ5BmfyfYChzM9tr8F8V,0.221,Tayo - Original,Gorge,0.000
4,0.002,https://api.spotify.com/v1/audio-analysis/3xYf...,0.734,407087,0.835,3xYfjgv22r16D8Vo8pBfuw,0.909,7,0.062,-9.677,...,0.039,124.994,4,https://api.spotify.com/v1/tracks/3xYfjgv22r16...,audio_features,spotify:track:3xYfjgv22r16D8Vo8pBfuw,0.810,Kotero Mi Amor,Shonky,0.000
5,0.049,https://api.spotify.com/v1/audio-analysis/6cLd...,0.710,517457,0.580,6cLdkAy1rv9eLn5IHYbif1,0.783,4,0.106,-11.876,...,0.038,120.002,4,https://api.spotify.com/v1/tracks/6cLdkAy1rv9e...,audio_features,spotify:track:6cLdkAy1rv9eLn5IHYbif1,0.418,Connaisseur - Original Mix,Olderic,0.000
6,0.003,https://api.spotify.com/v1/audio-analysis/5kG6...,0.810,379716,0.602,5kG6S9141LqnoUJDPMVzcm,0.894,2,0.048,-10.109,...,0.055,121.999,4,https://api.spotify.com/v1/tracks/5kG6S9141Lqn...,audio_features,spotify:track:5kG6S9141LqnoUJDPMVzcm,0.533,5000 Hz - Spieltape Remix,Spieltape,0.000
7,0.001,https://api.spotify.com/v1/audio-analysis/2cc2...,0.869,406000,0.589,2cc2WVBgDa6GafKhZEyDuL,0.883,1,0.055,-9.241,...,0.171,123.989,4,https://api.spotify.com/v1/tracks/2cc2WVBgDa6G...,audio_features,spotify:track:2cc2WVBgDa6GafKhZEyDuL,0.822,Microsociety,Alex Arnout,0.000
8,0.000,https://api.spotify.com/v1/audio-analysis/6q4O...,0.833,384733,0.486,6q4OyLRK01xAFTh6CRTUev,0.791,4,0.040,-9.665,...,0.075,123.034,4,https://api.spotify.com/v1/tracks/6q4OyLRK01xA...,audio_features,spotify:track:6q4OyLRK01xAFTh6CRTUev,0.645,My Boots,Gorge,0.000
9,0.003,https://api.spotify.com/v1/audio-analysis/505f...,0.894,467040,0.977,505fasbKly4AaDAESQ7bDN,0.556,7,0.129,-8.434,...,0.059,124.987,4,https://api.spotify.com/v1/tracks/505fasbKly4A...,audio_features,spotify:track:505fasbKly4AaDAESQ7bDN,0.748,Dance With Me,WestBoy,0.000
